**Verify Discharge**

Adrian Wiegman

2024-08-25

-------

## Description

This notebook turns raw data from LiDAR DEM, NHD flowlines, and polygons of cranberry bogs, into a layer of polygons representing the topographic catchments draining into each cranberry bog in southeast Massachusetts. 

Even though the lidar data as been hydro flattened/enforced, there are still a number of flowpaths that are not detected underneath highways (e.g. interstate I-495, I-195)

The solution is to "burn" in stream flowlines from the national hydrography dataset. This is done by putting a buffer around the stream flowline network, then assigning an arbitary large value to the stream network polygon, then converting to raster and subtracting the stream elevations from the DEM. 

Once D8 flow direction and flow accumulation rasters have been made the primary objectives can be completed. 

Data Sources:

1. Mass GIS Lidar DEM (1ft vertical resolution, ~1m horizontal resolution)
2. USGS National Hydrography Dataset
- Flow lines
3. Cranberry bogs layer

Steps of Processing: 

1. Prepare Lidar
   - 1M Lidar Elevation -> Clip to study area -> 
   - Resample to 10m resolution using aggregate minimum cell value
   - Fill Sinks 

2. Prepare Flowlines
   - combine flowlines into one layer
   - dissolve flowlines
   - buffer flowlines to 3x the resolution of processed lidar. 
       - buffering width of 15m 
   - convert to raster
3. Burn in flow lines
    - assign a value of -100 feet to flow lines
    - add flowlines to existing elevation (subtract 100 feet)
    
4. fill sinks (again)
5. D8 Flow Direction 
6. Flow Accumulation
7. Generate Bog Pour Points
    - find maximum flow accumulation value inside each bog.
    - generate a point at each bog maximum value.
    - identity to get cranberry bog attributes at each point.
8. Delineate Basins for each point
    - with the bog cranberry points loop through each point
        - delineate watershed using the bog pour point and the D8 flow direction
        - save the output to a temp file with the feature ID ('FID') of the cranberry bog. 
    - merge all cranberry bog basins to one polygon layer, containing the FID of the cranberry bog. 

## Setup Environment

In [1]:
# iphython options
# delete variables in workspace
%reset -f
#places plots inline
%matplotlib inline
#automatically reloads modules if they are changed
%load_ext autoreload 
%autoreload 2
# this codeblock sets up the environment from jupyter notebooks
setup_notebook = r"C:\Users\Adrian.Wiegman\Documents\GitHub\Wiegman_USDA_ARS\Cran_Q_C\2_gis\scripts\_Setup.ipynb"
%run $setup_notebook # magic command to run the notebook 

***
loading python modules...

  `module_list` contains names of all loaded modules

...module loading complete

***
loading user defined functions...

type `fn_`+TAB to for autocomplete suggestions

 the object `def_list` contains user defined function names:
   fn_get_info
   fn_arcgis_table_to_df
   fn_arcgis_table_to_np_to_pd_df
   fn_run_script_w_propy_bat
   fn_try_mkdir
   fn_hello
   fn_recursive_glob_search
   fn_regex_search_replace
   fn_regex_search_0
   fn_arcpy_table_to_excel
   fn_agg_sum_df_on_group
   fn_add_prefix_suffix_to_selected_cols
   fn_calc_pct_cover_within_groups
   fn_buildWhereClauseFromList
   fn_FA_to_Q
   fn_alter_field_double
   fn_return_float
   fn_classify_wetlands

 use ??{insert fn name} to inspect
 for example running `??fn_get_info` returns:
Signature: fn_get_info(name='fn_get_info')
Source:   
def fn_get_info(name='fn_get_info'):
    '''
    returns the source information about a given function name
    '''
    get_ipython().run_line_magic('pinf

In [52]:
# Extract bog Q and C information for flow verification bogs. 
bogs = "bogs_agg_verify_Q"
arcpy.management.SelectLayerByAttribute(
    in_layer_or_view=bogs,
    selection_type="CLEAR_SELECTION",
    where_clause="",
    invert_where_clause=None)
joinout = r"C:\Workspace\Geodata\Cran_Q_C\Default.gdb\bogs_agg_verify_extract_Q_C"
arcpy.gapro.JoinFeatures(
    target_layer="bogs_agg_verify_Q",
    join_layer=r"C:\Workspace\Geodata\Cran_Q_C\Default.gdb\bogs_pour_points_ident_agg_raw_MEP",
    output=joinout,
    join_operation="JOIN_ONE_TO_MANY",
    spatial_relationship="INTERSECTS",
    spatial_near_distance=None,
    temporal_relationship="",
    temporal_near_distance=None,
    attribute_relationship=None,
    summary_fields=None,
    join_condition="",
    keep_all_target_features=None,
    include_distance=None,
    distance_unit="")
arcpy.conversion.TableToExcel(
    Input_Table=joinout,
    Output_Excel_File=os.path.join(r"C:\Workspace\Geodata\Cran_Q_C\outputs","bogs_agg_verify_extract_Q_C.xls"),
    Use_field_alias_as_column_header="NAME",
    Use_domain_and_subtype_description="CODE")

<Result 'C:\\Workspace\\Geodata\\Cran_Q_C\\outputs\\bogs_agg_verify_extract_Q_C.xls'>

In [53]:
# 8/27/24
# locations were edited on  to MEP snap monitoring locations to flow lines where there were GPS location errors
# - Scorton Creek at Jones Ln
# - Little Creek
# two additional sites were added from the MEP Barnstable Report, where zero flow was recorded
# - Scorton Creek 
inpoints = r"df_Q_bogs_streams_XYTableToPoint_edit"
arcpy.management.SelectLayerByAttribute(
    in_layer_or_view=inpoints,
    selection_type="CLEAR_SELECTION",
    where_clause="",
    invert_where_clause=None)

# make a copy to work from 
points = inpoints+"_extract_Q_C"
arcpy.management.CopyFeatures(
    in_features=inpoints,
    out_feature_class=points,
    config_keyword="",
    spatial_grid_1=None,
    spatial_grid_2=None,
    spatial_grid_3=None)

<Result 'C:\\Workspace\\Geodata\\Cran_Q_C\\Default.gdb\\df_Q_bogs_streams_XYTableToPoint_edit_extract_Q_C'>

In [54]:
arcpy.management.JoinField(
    in_data=points,
    in_field="OBJECTID",
    join_table=bogs,
    join_field="FID_df_Q_bogs_streams",
    fields="FID_bogs_agg;site_id;ORIG_FID",
    fm_option="NOT_USE_FM",
    field_mapping=None,
    index_join_fields="NO_INDEXES"
)

<Result 'C:\\Workspace\\Geodata\\Cran_Q_C\\Default.gdb\\df_Q_bogs_streams_XYTableToPoint_edit_extract_Q_C'>

In [55]:
arcpy.analysis.Buffer(
    in_features=points,
    out_feature_class=r"df_Q_bogs_streams_XY_buffer150m",
    buffer_distance_or_field="150 Meters",
    line_side="FULL",
    line_end_type="ROUND",
    dissolve_option="NONE",
    dissolve_field=None,
    method="GEODESIC"
)

<Result 'C:\\Workspace\\Geodata\\Cran_Q_C\\Default.gdb\\df_Q_bogs_streams_XY_buffer150m'>

In [56]:
names = ["FA_D8_gwe_f",
         "FA_D8_gwe_bf",
         "FA_D8_gwe_bf_lt1m",
         #"FA_MFD_gwe_f",
         "FA_MFD_gwe_bf",
         #"FA_MFD_gwe_bf_lt1m",
         #"FA_Dinf_gwe_f",
         "FA_Dinf_gwe_bf",
         #"FA_Dinf_gwe_bf_lt1m",
         # 08/24/2024
         "FA_D8_gwe_lid_lt1m",
         "FA_D8_gwe_lid_lt0m",
         # 08/28/24
         # 08/29/24
"FA_D8_lidbr", #lid - burn_flowline_raw
"FA_D8_lidb", #lid - burn_flowline
"FA_D8_lidbc", #lid - burn_combined
"FA_D8_lidbcr", #lid - burn_combined
"FA_D8_gwebr", #gwe - burn_flowline_raw
"FA_D8_gweb", #gwe - burn_flowline
"FA_D8_gwebc", #gwe - burn_combined
"FA_D8_gwebcr", #gwe - burn_combined_raw
"FA_D8_gweblidbk1", #gweb*(1-0.01) + lidb*0.01
"FA_D8_gwebclidbck1", #gwebc*(1-0.01) + lidbc*0.01
"FA_D8_gweblidbk5", #gweb*(1-0.05) + lidb*0.05
"FA_D8_gwebrlidbrk1", #gwebr*(1-0.01) + lidbr*0.01
"FA_D8_gwebcrlidbcrk1", #gwebcr*(1-0.01) + lidbcr*0.01
"FA_D8_gwebrlidbrk5", #gwebr*(1-0.05) + lidbr*0.05
"FA_D8_gwebcrlidbcrk5", #gwebcr*(1-0.05) + lidbcr*0.05
"FA_D8_gweblidbk1_lid1", #(gweblidbk1 + 1)*gwd_ge1 + lidb*gwd_lt1 - 1 - burn_flowline
"FA_D8_gweblidbk1_lid0", #(gweblidbk1 + 0)*gwd_ge1 + lidb*gwd_lt0 - 0 - burn_flowline
"FA_D8_gweblidbk5_lid1", #(gweblidbk5 + 1)*gwd_ge1 + lidb*gwd_lt1 - 1 - burn_flowline
"FA_D8_gweblidbk5_lid0", #(gweblidbk5 + 0)*gwd_ge1 + lidb*gwd_lt0 - 0 - burn_flowline
"FA_D8_gwelidr1", #(gwebr + 1)*gwd_ge1 + lidbr*gwd_lt1 - 1 - burn_flowline
"FA_D8_gwelidr0", #(gwebr + 0)*gwd_ge0 + lidbr*gwd_lt0 - 0 - burn_flowline
"FA_D8_gwelid1", #(gweb + 1)*gwd_ge1 + lidb*gwd_lt1 - 1 - burn_flowline
"FA_D8_gwelid0", #(gweb + 0)*gwd_ge0 + lidb*gwd_lt0 - 0 - burn_flowline
        ]
files = [os.path.join(gdb_default_path,n) for n in names]
print(files)

['C:\\Workspace\\Geodata\\Cran_Q_C\\Default.gdb\\FA_D8_gwe_f', 'C:\\Workspace\\Geodata\\Cran_Q_C\\Default.gdb\\FA_D8_gwe_bf', 'C:\\Workspace\\Geodata\\Cran_Q_C\\Default.gdb\\FA_D8_gwe_bf_lt1m', 'C:\\Workspace\\Geodata\\Cran_Q_C\\Default.gdb\\FA_MFD_gwe_bf', 'C:\\Workspace\\Geodata\\Cran_Q_C\\Default.gdb\\FA_Dinf_gwe_bf', 'C:\\Workspace\\Geodata\\Cran_Q_C\\Default.gdb\\FA_D8_gwe_lid_lt1m', 'C:\\Workspace\\Geodata\\Cran_Q_C\\Default.gdb\\FA_D8_gwe_lid_lt0m', 'C:\\Workspace\\Geodata\\Cran_Q_C\\Default.gdb\\FA_D8_lidbr', 'C:\\Workspace\\Geodata\\Cran_Q_C\\Default.gdb\\FA_D8_lidb', 'C:\\Workspace\\Geodata\\Cran_Q_C\\Default.gdb\\FA_D8_lidbc', 'C:\\Workspace\\Geodata\\Cran_Q_C\\Default.gdb\\FA_D8_lidbcr', 'C:\\Workspace\\Geodata\\Cran_Q_C\\Default.gdb\\FA_D8_gwebr', 'C:\\Workspace\\Geodata\\Cran_Q_C\\Default.gdb\\FA_D8_gweb', 'C:\\Workspace\\Geodata\\Cran_Q_C\\Default.gdb\\FA_D8_gwebc', 'C:\\Workspace\\Geodata\\Cran_Q_C\\Default.gdb\\FA_D8_gwebcr', 'C:\\Workspace\\Geodata\\Cran_Q_C\\Default.

In [57]:
in_feat = "df_Q_bogs_streams_XY_buffer150m"
for i in range(len(files)):
    out_raster = arcpy.ia.ZonalStatistics(
        in_zone_data=in_feat,
        zone_field="OBJECTID",
        in_value_raster=files[i],
        statistics_type="MAXIMUM",
        ignore_nodata="DATA",
        process_as_multidimensional="CURRENT_SLICE",
        percentile_value=90,
        percentile_interpolation_type="AUTO_DETECT",
        circular_calculation="ARITHMETIC",
        circular_wrap_value=360)
    out_rast_name = "ZS_MAX_verify_"+names[i]
    out_raster.save(out_rast_name)
    

In [58]:
# generate string of rasters to 

long_names = ["ZS_MAX_verify_"+n for n in names]
long_names
_ = ["{} {}".format(os.path.join(gdb_default_path,n),n) for n in long_names]
in_rasters = ";".join(_)
print(in_rasters)

C:\Workspace\Geodata\Cran_Q_C\Default.gdb\ZS_MAX_verify_FA_D8_gwe_f ZS_MAX_verify_FA_D8_gwe_f;C:\Workspace\Geodata\Cran_Q_C\Default.gdb\ZS_MAX_verify_FA_D8_gwe_bf ZS_MAX_verify_FA_D8_gwe_bf;C:\Workspace\Geodata\Cran_Q_C\Default.gdb\ZS_MAX_verify_FA_D8_gwe_bf_lt1m ZS_MAX_verify_FA_D8_gwe_bf_lt1m;C:\Workspace\Geodata\Cran_Q_C\Default.gdb\ZS_MAX_verify_FA_MFD_gwe_bf ZS_MAX_verify_FA_MFD_gwe_bf;C:\Workspace\Geodata\Cran_Q_C\Default.gdb\ZS_MAX_verify_FA_Dinf_gwe_bf ZS_MAX_verify_FA_Dinf_gwe_bf;C:\Workspace\Geodata\Cran_Q_C\Default.gdb\ZS_MAX_verify_FA_D8_gwe_lid_lt1m ZS_MAX_verify_FA_D8_gwe_lid_lt1m;C:\Workspace\Geodata\Cran_Q_C\Default.gdb\ZS_MAX_verify_FA_D8_gwe_lid_lt0m ZS_MAX_verify_FA_D8_gwe_lid_lt0m;C:\Workspace\Geodata\Cran_Q_C\Default.gdb\ZS_MAX_verify_FA_D8_lidbr ZS_MAX_verify_FA_D8_lidbr;C:\Workspace\Geodata\Cran_Q_C\Default.gdb\ZS_MAX_verify_FA_D8_lidb ZS_MAX_verify_FA_D8_lidb;C:\Workspace\Geodata\Cran_Q_C\Default.gdb\ZS_MAX_verify_FA_D8_lidbc ZS_MAX_verify_FA_D8_lidbc;C:\Workspa

In [59]:
arcpy.sa.ExtractMultiValuesToPoints(
    in_point_features=points,
    in_rasters=in_rasters,
    bilinear_interpolate_values="NONE")

<geoprocessing server result object object at 0x000002926F0C92A0>

FA = Flow accumulation (number of cells draining to a point)

w = width of cell in distance units (10 meters)

l = length of cell in distance units (10 meters)

A = FA * cell_size^2

cell size is the side length of the grid cells

A = FA * 100 m^2/cell

Q = A * r

where r is recharge rate 
27.25 in/yr 

m3/d =  27.25 in/yr * 2.54 cm/in * 1/100 m/cm *  1/365.25 yr/d

In [60]:
??fn_FA_to_Q

Signature: fn_FA_to_Q(rasterpath=None, recharge_rate_in_yr=27.25)
Docstring: <no docstring>
Source:   
def fn_FA_to_Q (rasterpath=None,recharge_rate_in_yr = 27.25):
    _ = arcpy.GetRasterProperties_management(rasterpath, "CELLSIZEX")
    #Get the elevation standard deviation value from geoprocessing result object
    cellsize_x = _.getOutput(0)
    _ = arcpy.GetRasterProperties_management(rasterpath, "CELLSIZEY")
    cellsize_y = _.getOutput(0)
    # calculate cell area in meters
    cell_area_meters = float(cellsize_x) * float(cellsize_y)
    print("cell area {} square meters".format(cell_area_meters))
    FA_to_Q = cell_area_meters * recharge_rate_in_yr * 2.54 * (1/100) * (1/365.25)
    print("FA_to_Q = {} m3/d per cell".format(FA_to_Q))
    return(FA_to_Q)
File:      c:\windows\system32\<ipython-input-1-ca1a1ecab51d>
Type:      function


In [61]:
fn_FA_to_Q(rasterpath=os.path.join(gdb_default_path,"FA_D8_gwe_bf"))

cell area 100.0 square meters
FA_to_Q = 0.1895003422313484 m3/d per cell


0.1895003422313484

In [62]:
# calculate flow for all flow accumulation layers
# r"C:\Workspace\Geodata\Verify_Discharge\Verify_Discharge.gdb\bogs_points"
rgx = "ZS_MAX_verify_(.*)"
table_name = points
in_table = os.path.join(gdb_default_path,table_name)
arcpy.management.SelectLayerByAttribute(
        in_layer_or_view=in_table,
        selection_type="CLEAR_SELECTION")

<Result 'df_Q_bogs_streams_XYTableToP2'>

In [64]:
%%capture 
# supress outputs
import warnings
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    for long_name in long_names:
        print(long_name)
        FA_to_Q = fn_FA_to_Q(rasterpath=os.path.join(gdb_default_path,long_name))
        _ = re.search(rgx,long_name)
        short_name = re.sub("FA","",re.sub("_","",_[1]))
        print(short_name)
        arcpy.management.SelectLayerByAttribute(
            in_layer_or_view=in_table,
            selection_type="NEW_SELECTION",
            where_clause="{} IS NOT NULL".format(long_name))
        arcpy.management.CalculateField(
            in_table=in_table,
            field="Q_m3d_{}".format(short_name),
            expression="!{}!*{}".format(long_name,FA_to_Q),
            expression_type="PYTHON3",
            code_block="",
            field_type="DOUBLE",
            enforce_domains="NO_ENFORCE_DOMAINS")
        arcpy.management.SelectLayerByAttribute(
            in_layer_or_view=in_table,
            selection_type="CLEAR_SELECTION")

In [66]:
names = ["EBK_NO3_p25",
         "EBK_NO3_p50",
         "EBK_NO3_p75",
         "EBK_TN_p25",
         "EBK_TN_p50",
         "EBK_TN_p75",]
files = [os.path.join(gdb_default_path,n) for n in names]
print(files)

['C:\\Workspace\\Geodata\\Cran_Q_C\\Default.gdb\\EBK_NO3_p25', 'C:\\Workspace\\Geodata\\Cran_Q_C\\Default.gdb\\EBK_NO3_p50', 'C:\\Workspace\\Geodata\\Cran_Q_C\\Default.gdb\\EBK_NO3_p75', 'C:\\Workspace\\Geodata\\Cran_Q_C\\Default.gdb\\EBK_TN_p25', 'C:\\Workspace\\Geodata\\Cran_Q_C\\Default.gdb\\EBK_TN_p50', 'C:\\Workspace\\Geodata\\Cran_Q_C\\Default.gdb\\EBK_TN_p75']


In [67]:
arcpy.sa.ExtractMultiValuesToPoints(
    in_point_features=points,
    in_rasters=files,
    bilinear_interpolate_values="NONE")

<geoprocessing server result object object at 0x00000292856B2540>

In [68]:
print(points)

df_Q_bogs_streams_XYTableToPoint_edit_extract_Q_C


In [69]:
arcpy.conversion.TableToExcel(
    Input_Table=points,
    Output_Excel_File=os.path.join(r"C:\Workspace\Geodata\Cran_Q_C\outputs",points+".xls"),
    Use_field_alias_as_column_header="NAME",
    Use_domain_and_subtype_description="CODE")

<Result 'C:\\Workspace\\Geodata\\Cran_Q_C\\outputs\\df_Q_bogs_streams_XYTableToPoint_edit_extract_Q_C.xls'>

# Appendix 

unused code

## Delineate watersheds